<a href="https://colab.research.google.com/github/Hasama-Twins/DeepLearningHaiku/blob/main/DeepLearningHaiku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import and Process Data



Add this [folder](https://drive.google.com/drive/folders/1xxLhcOfR1TtFq7p7ORBJibDP3ykvpnyn?usp=sharing) to your Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

In [5]:
# Sourced from https://www.kaggle.com/datasets/bfbarry/haiku-dataset
lines_df = pd.read_csv('/content/drive/MyDrive/DeepLearningHaikuFolder/lines.txt', sep="/", header=None)
lines_df

,0,1,2
0,delicate savage,you'll never hold the cinder,but still you will burn $
1,our destination,the skyline of this city,shining horizon $
2,a splash and a cry,words pulled from the riverside,dried in the hot sun $
3,hurt but poised for war,sturdy in crestfallen slumps,warrior spirit $
4,steamy mist rising,rocks receiving downward crash,as the jungle weeps $
...,...,...,...
11264,mods removing posts,is reminiscent of days,gone by book burnings $
11265,an otter went down,into the glassy water,to catch her breakfast $
11266,do not waste your time,and know that without failure,you cannot improve $
11267,begging my way back,another night on this road,i gaze at the moon $


In [6]:
without_dollar_sign = lines_df[2].str.rstrip('$')
lines_df[2]=without_dollar_sign
lines_df

,0,1,2
0,delicate savage,you'll never hold the cinder,but still you will burn
1,our destination,the skyline of this city,shining horizon
2,a splash and a cry,words pulled from the riverside,dried in the hot sun
3,hurt but poised for war,sturdy in crestfallen slumps,warrior spirit
4,steamy mist rising,rocks receiving downward crash,as the jungle weeps
...,...,...,...
11264,mods removing posts,is reminiscent of days,gone by book burnings
11265,an otter went down,into the glassy water,to catch her breakfast
11266,do not waste your time,and know that without failure,you cannot improve
11267,begging my way back,another night on this road,i gaze at the moon


In [7]:
lines_df.columns = ["0","1","2"]

<ipython-input-7-15185f3ffadd>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = lines_df.append(all_haiku_df[["0","1","2"]], ignore_index = True)


,0,1,2
0,delicate savage,you'll never hold the cinder,but still you will burn
1,our destination,the skyline of this city,shining horizon
2,a splash and a cry,words pulled from the riverside,dried in the hot sun
3,hurt but poised for war,sturdy in crestfallen slumps,warrior spirit
4,steamy mist rising,rocks receiving downward crash,as the jungle weeps
...,...,...,...
155387,I'm not asking did,you say it nor clarify,what you said neither
155388,You are truly a,moron or a liar I'm,inclined to think both
155389,Ain't no selfie on,this earth that's gonna make me,like Theresa May
155390,is doing a great,job turning Independents,into Democrats


In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2') # https://www.sbert.net/

In [10]:
# embed each line of the poem and take the mean
def encode(sentence):
  return np.mean(model.encode(sentence))

vector_df = lines_df.applymap(encode)

In [13]:
vector_df

,0,1,2
0,0.001474,0.002309,0.001961
1,0.000581,-0.000074,0.000566
2,0.000029,-0.000322,0.000685
3,-0.000037,-0.000589,0.001221
4,0.000635,0.001094,0.001092
...,...,...,...
11264,0.000355,0.000800,0.001194
11265,-0.000332,0.001965,0.000962
11266,0.000650,0.001259,0.002232
11267,0.000342,0.000298,0.000459


In [ ]:
!pip install tensorflow

In [ ]:
!pip install imageai --upgrade

In [40]:
from imageai.Classification import ImageClassification
import os

# https://imageai.readthedocs.io/en/latest/prediction/index.html

path = '/content/drive/MyDrive/DeepLearningHaikuFolder/'
prediction = ImageClassification()
prediction.setModelTypeAsResNet50()
prediction.setModelPath(path+ "resnet50-19c8e357.pth")
prediction.loadModel()

# get 3 keywords from each image 

# Sourced from https://www.kaggle.com/datasets/puneet6060/intel-image-classification

images_path = '/content/drive/MyDrive/DeepLearningHaikuFolder/images'
image_keywords_df = pd.DataFrame(columns=['kw1', 'kw2', 'kw3'])

for i, filename in enumerate(os.listdir(images_path)):
    if filename.endswith("jpg"): 
        predictions, probabilities = prediction.classifyImage(images_path+"/"+filename, result_count=3)
        image_keywords_df.loc[i] = predictions

image_keywords_df.head
        

<bound method NDFrame.head of                kw1         kw2            kw3
0          volcano         alp     promontory
1              alp      valley  mountain tent
2           geyser         alp        dogsled
3    mountain tent         alp       lakeside
4             plow  park bench        mailbox
..             ...         ...            ...
995      speedboat   catamaran       trimaran
996        gondola   streetcar            cab
997         prison   monastery          vault
998            alp      valley            ski
999       lakeside    seashore          wreck

[1000 rows x 3 columns]>

In [43]:
X = image_keywords_df.copy() # the keywords taken from image
Y = vector_df.copy() # the mean embeddings for each line of the haiku's